# G only ACGT Probabilities

> 

In [ ]:
import os, json, re

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import hilbertcurve
from hilbertcurve.hilbertcurve import HilbertCurve

from EnvDL.core import * # includes remove_matching_files
from EnvDL.dna  import *
from EnvDL.dlfn import * # includes LSUV_

from tqdm import tqdm

In [ ]:
use_gpu_num = 0

# Imports --------------------------------------------------------------------
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F # F.mse_loss

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger


device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [ ]:
cache_path = '../nbs_artifacts/02.20_g2fc_G_ACGT_FCN/'
ensure_dir_path_exists(dir_path = cache_path)

In [ ]:
# import os, re, json
# from tqdm import tqdm

# import numpy as np
# import pandas as pd
# pd.set_option('display.max_columns', None)

# import plotly.express as px
# import plotly.io as pio
# pio.templates.default = "plotly_white"

# import torch
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
# from torch import nn

# from EnvDL.core import * 
# # includes 
#     # remove_matching_files 
#     # read_json
# from EnvDL.dna import *
# from EnvDL.dlfn import * 
# # includes
#     # read_split_info 
#     # find_idxs_split_dict
#     # train_loop_yx
#     # train_error_yx
#     # test_loop_yx
#     # train_nn_yx
#     # yhat_loop_yx

# from tqdm import tqdm

In [ ]:
# use_gpu_num = 0

# device = "cuda" if torch.cuda.is_available() else "cpu"
# if use_gpu_num in [0, 1]: 
#     torch.cuda.set_device(use_gpu_num)
# print(f"Using {device} device")

In [ ]:
# # Imports for the Adaptive Experimentation (Ax) platform
# from ax.service.ax_client import AxClient, ObjectiveProperties
# from ax.utils.measurement.synthetic_functions import hartmann6
# from ax.utils.notebook.plotting import render, init_notebook_plotting

# import pickle as pkl
# # Because the search space is nested, it's not storable with the builtin methods for saving/reading json (not tested with sql)
# # pickling seems to work just fine.

# init_notebook_plotting()

In [ ]:
# cache_path = '../nbs_artifacts/02.20_g2fc_G_ACGT_FCN/'
# ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [ ]:
load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
phno_geno = pd.read_csv(load_from+'phno_geno.csv')
phno = phno_geno

obs_geno_lookup = np.load(load_from+'obs_geno_lookup.npy') # Phno_Idx	Geno_Idx	Is_Phno_Idx
YMat = np.load(load_from+'YMat.npy')
ACGT = np.load(load_from+'ACGT.npy')

### Create train/test validate indicies from json

In [ ]:
load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/'

split_info = read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26')

temp = phno.copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

test_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['test'][0]
)

temp = temp.loc[test_dict['train_idx'], ] # restrict before re-aplying

val_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['validate'][0]
)

In [ ]:
train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

In [ ]:
# confirm all observation idxs are have genomic information
assert [] == [e for e in list(train_idx)+list(test_idx) if e not in obs_geno_lookup[:, 0]]

## One Hot Encoded

In [ ]:
# confirm all observation idxs are have genomic information
# assert [] == [e for e in list(train_idx)+list(test_idx) if e not in obs_geno_lookup[:, 0]]

In [ ]:
YMat_cs = calc_cs(YMat[train_idx])
y_cs = apply_cs(YMat, YMat_cs)

In [ ]:
# Debugging Data -----------------------------------------------------------------------------------

train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

training_dataloader = DataLoader(
    ACGTDataset(y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float), 
                G = torch.from_numpy(ACGT).to(torch.float), 
                idx_original = torch.from_numpy(np.array(train_idx)),
                idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
                use_gpu_num = 0,
                device = 'cuda'
               ),
    batch_size = 50,
    shuffle = True
)

validation_dataloader = DataLoader(
    ACGTDataset(y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float), 
                G = torch.from_numpy(ACGT).to(torch.float), 
                idx_original = torch.from_numpy(np.array(test_idx)),
                idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
                use_gpu_num = 0,
                device = 'cuda'
               ),
    batch_size = 50,
    shuffle = True
)

In [ ]:
# A quirk of this is that to get only a single layer the length of the input tensor must be passed in. for 2+ I'll figure it out.
class NeuralNetwork(nn.Module):
    def __init__(self, parameterization):
        super(NeuralNetwork, self).__init__()            
        module_list = []

        max_layer = parameterization['num_layers']
        for i in range(max_layer):
            if i  == 0:
                name_in = f"in_{i+1}_of_{max_layer}"
            else:
                name_in = f"out_{i}_of_{max_layer}"
            name_out = f"out_{i+1}_of_{max_layer}"
            name_drop= f"drop_{i+1}_of_{max_layer}"

            if i == 0:
                module_list += [nn.Flatten()]
            

            module_list += [
                Linear_block(
                    in_size  = parameterization[name_in], 
                    out_size = parameterization[name_out], 
                    drop_pr  = parameterization[name_drop])]
            
            if (i+1) == max_layer:
                module_list += [nn.Linear(parameterization[name_out], 1)]
                
        self.x_network = nn.ModuleList(module_list)
        
    def forward(self, x):
        
        for mod in self.x_network:
            if mod == self.x_network[-1]:
                out = x # get the penultimate layer's outputs for later
            x = mod(x)
        
        pred = x
        return pred, out

# model = NeuralNetwork(
#     parameterization = parameterization).to(device)

# model(next(iter(training_dataloader))[0][0:5])

In [ ]:
# convenince wrapper to fill in for R's seq or x:y notation
def linrange(start, stop):
    import numpy as np
    diff = start - stop
    res = np.linspace(start, stop, abs(diff)+1).astype(int)
    return res 

In [ ]:
            # [2048, 1024, 512, 256, 128, 64, 32, 16]
layer_sizes = [int(2**i) for i in linrange(start = 11, stop = 4)]
layer_drops = [0.1 for e in layer_sizes]

num_layers = len(layer_sizes)

params = {
    'num_layers':num_layers,
    f"in_1_of_{num_layers}": (ACGT.shape[1] * ACGT.shape[2])
}

for i in range(num_layers):
    params[f"out_{ i + 1}_of_{num_layers}"] = layer_sizes[i]
    params[f"drop_{ i + 1}_of_{num_layers}"] = layer_drops[i]
        

In [ ]:
model = NeuralNetwork(parameterization = params).to(device)
# model

In [ ]:
LSUV_(model, data = next(iter(training_dataloader))[0] )

Applying orthogonal init (zero init if dim < 2) to params in 9 module(s).
Applying LSUV to 9 module(s) (up to 10 iters per module):
Module  0 after  2 itr(s) | Mean:  0.041 | Std: 1.000 | <class 'torch.nn.modules.linear.Linear'>
Module  1 after  2 itr(s) | Mean: -0.049 | Std: 0.994 | <class 'torch.nn.modules.linear.Linear'>
Module  2 after  2 itr(s) | Mean:  0.044 | Std: 0.999 | <class 'torch.nn.modules.linear.Linear'>
Module  3 after  2 itr(s) | Mean:  0.020 | Std: 1.002 | <class 'torch.nn.modules.linear.Linear'>
Module  4 after  2 itr(s) | Mean:  0.032 | Std: 1.038 | <class 'torch.nn.modules.linear.Linear'>
Module  5 after  2 itr(s) | Mean: -0.114 | Std: 0.998 | <class 'torch.nn.modules.linear.Linear'>
Module  6 after  2 itr(s) | Mean: -0.113 | Std: 0.992 | <class 'torch.nn.modules.linear.Linear'>
Module  7 after  2 itr(s) | Mean: -0.203 | Std: 0.951 | <class 'torch.nn.modules.linear.Linear'>
Module  8 after  2 itr(s) | Mean: -0.444 | Std: 0.954 | <class 'torch.nn.modules.linear.Line

In [ ]:
# model(next(iter(training_dataloader))[0])

In [ ]:
# Module for training subnetworks.
class plDNN_ACGT(pl.LightningModule):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        
    def training_step(self, batch, batch_idx):
        g_i, y_i = batch
        pred, out = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        
        with torch.no_grad():
            weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
            for l in weight_list:
                self.log(("train_mean"+l[0]), l[1].mean())
                self.log(("train_std"+l[0]), l[1].std())        
        return(loss)
        
    def validation_step(self, batch, batch_idx):
        g_i, y_i = batch
        pred, out = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    

In [ ]:
max_epoch = 100
DNNG = plDNN_ACGT(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-fcn")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: tb_logs/g-acgt-fcn
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | mod  | NeuralNetwork | 1.0 B 
---------------------------------------
1.0 B     Trainable params
0         Non-trainable params
1.0 B     Total params
4,136.397 Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
torch.save(DNNG.mod, cache_path+'g-acgt-fcn'+'.pt')

In [ ]:
# ACGTDataset

In [ ]:
# class DNNCO_G(nn.Module):
#     def __init__(self):
#         super(DNNCO_G, self).__init__()    
        
#         self.g_network =nn.Sequential(
#             Linear_block(in_size= 4881, out_size= 83, drop_pr= 0.163923177),
#             Linear_block(in_size= 83,   out_size= 133, drop_pr= 0.230663142)
#         ) 
#         self.g_pred =nn.Sequential(
#             nn.Linear(133, 1)
#         ) 
                
#     def forward(self, tensor_list):
#         y, g, s, w = tensor_list
#         g_out = self.g_network(g)
#         g_pred = self.g_pred(g_out)
#         return g_pred, g_out
    
# DNNCO_G().to(device)(next(iter(training_dataloader)) )[0].shape

# model = DNNCO_G().to(device)
# LSUV_(model, data = next(iter(training_dataloader)) )

In [ ]:
# # Module for training subnetworks.
# class plDNNCO_subnet(pl.LightningModule):
#     def __init__(self, mod):
#         super().__init__()
#         self.mod = mod
        
#     def training_step(self, batch, batch_idx):
#         # train loop
# #         y_i, g_i, s_i, w_i = batch
# #         pred, out = self.mod(g_i, s_i, w_i)
#         y_i, _, _, _ = batch
#         pred, out = self.mod(batch)
#         # print(y_i.shape, pred.shape)
#         loss = F.mse_loss(pred, y_i)
#         self.log("train_loss", loss)
        
#         with torch.no_grad():
#             weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
#             for l in weight_list:
#                 self.log(("train_mean"+l[0]), l[1].mean())
#                 self.log(("train_std"+l[0]), l[1].std())        
#         return(loss)
        
#     def validation_step(self, batch, batch_idx):
# #         y_i, g_i, s_i, w_i = batch
# #         pred, out = self.mod(g_i, s_i, w_i)
#         y_i, _, _, _ = batch
#         pred, out = self.mod(batch)
#         # print(y_i.shape, pred.shape)
#         loss = F.mse_loss(pred, y_i)
#         self.log('val_loss', loss)        
     
#     def configure_optimizers(self, **kwargs):
#         optimizer = torch.optim.Adam(self.parameters(), **kwargs)
#         return optimizer    

## Tune with `ax`

In [ ]:
# needs to be able to take a dictionary with the parameters that can be varied.

# must subclass train_nn to make changes to the optimizer

In [ ]:
# xs = [i for i in np.linspace(1, 32, 10)]
# ys = [round(e**2) for e in xs]
# px.scatter(
#     pd.DataFrame({'x': xs, 'y': ys}),
#     x ='x', y= 'y')

# [int(2**i) for i in np.linspace(1, 10, 10)]

In [ ]:
# num_layer_blocks = [1]+[2*(1+i) for i in range(6)]
# in_size_int = 4*125891
# out_size_choice = [2**(4+i) for i in range(7)] #16 <-> 1024
# drop_pr_choice = [(5*e)/100 for e in range(11)] # 0 <-> 50
# epoch_choice = [2, 1024]

# # See https://github.com/facebook/Ax/issues/1454#issuecomment-1462417398

# # num_layer_blocks = [1, 2]
# # in_size_int
# # out_size_choice = [16, 32]
# # drop_pr_choice = [0.0, 0.3]
# # epoch_choice

# params = [
#     {
#         "name": "num_layers",
#         "type": "choice",
#         "is_ordered": True, 
#         "values": num_layer_blocks,
#         "dependents": {
#             num_layers: [
#                 f"in_{  i + 1}_of_{num_layers}" for i in range(num_layers) if i == 0]+[
#                 f"out_{ i + 1}_of_{num_layers}" for i in range(num_layers)]+[
#                 f"drop_{i + 1}_of_{num_layers}" for i in range(num_layers)]+[
#                 f"epoch_of_{num_layers}" for i in range(num_layers) if i == 0
#             ] for num_layers in num_layer_blocks
#         }
#     },
#     *[  # * here puts the entries of the list comprehension into the outer list instead of keeping them as a sub-list
#         {
#             "name": f"in_{ i + 1}_of_{num_layers}",
#             "type": "fixed",
#             "value_type": 'int',
#             "value": in_size_int
#         } for num_layers in num_layer_blocks for i in range(num_layers) if i == 0
        
#     ],
#     *[
#         {
#             "name": f"out_{ i + 1}_of_{num_layers}",
#             "type": "choice",
#             "is_ordered": True, 
#             "value_type": 'int',
#             "values": out_size_choice
#         } for num_layers in num_layer_blocks for i in range(num_layers) 
        
#     ],
#     *[
#         {
#             "name": f"drop_{ i + 1}_of_{num_layers}",
#             "type": "choice",
#             "is_ordered": True, 
#             "value_type": 'float',
#             "values": drop_pr_choice
#         } for num_layers in num_layer_blocks for i in range(num_layers) 
        
#     ],
#     *[
#         {
#             "name": f"epoch_of_{num_layers}",
#             "type": "range",
# #             "is_ordered": True, 
#             "value_type": 'int',
#             "bounds": epoch_choice
#         } for num_layers in num_layer_blocks 
        
#     ],
    
# ]
# # params

In [ ]:
# # Initialize
# ax_client = AxClient()

# # Using SAASBO per https://github.com/facebook/Ax/issues/1454#issuecomment-1462417398
# # Create an experiment with required arguments: name, parameters, and objective_name.
# ax_client.create_experiment(
#     name="tune_fcn_layer_number",  # The name of the experiment.
#     parameters=params,
#     objectives={"rmse": ObjectiveProperties(minimize=True)},  # The objective name and minimization setting.
#     # parameter_constraints: Optional, a list of strings of form "p1 >= p2" or "p1 + p2 <= some_bound".
#     # outcome_constraints: Optional, a list of strings of form "constrained_metric <= some_bound".
#     choose_generation_strategy_kwargs={"use_saasbo": True}
# )

In [ ]:
# # if there already is a set of test results, then load it.
# if os.path.exists(cache_path+'ax_client.pkl'):
#     ax_client = None
#     with open(cache_path+'ax_client.pkl', 'rb') as f:
#         dat = pkl.load(f)
#     ax_client = dat

In [ ]:
# class NeuralNetwork(nn.Module):
#     def __init__(self, parameterization):
#         super(NeuralNetwork, self).__init__()    

#         def Linear_block(in_size, out_size, drop_pr):
#             block = nn.Sequential(
#                 nn.Linear(in_size, out_size),
#                 nn.ReLU(),
#                 nn.Dropout(drop_pr)
#             )
#             return(block)         
        
#         module_list = []

#         max_layer = parameterization['num_layers']
#         for i in range(max_layer):
#             if i  == 0:
#                 name_in = f"in_{i+1}_of_{max_layer}"
#             else:
#                 name_in = f"out_{i}_of_{max_layer}"
#             name_out = f"out_{i+1}_of_{max_layer}"
#             name_drop= f"drop_{i+1}_of_{max_layer}"

#             if i == 0:
#                 module_list += [nn.Flatten()]

#             module_list += [
#                     Linear_block(
#                         in_size  = parameterization[name_in], 
#                         out_size = parameterization[name_out], 
#                         drop_pr  = parameterization[name_drop])]

#             if (i+1) == max_layer:
#                 module_list += [nn.Linear(parameterization[name_out], 1)]
                
#         self.x_network = nn.ModuleList(module_list)
        
#     def forward(self, x):
        
#         for mod in self.x_network:
#             x = mod(x)
            
#         return x

# # model = NeuralNetwork(
# #     parameterization = parameterization).to(device)

# # model(next(iter(training_dataloader))[0][0:5])

In [ ]:
# Get the parameters and run the trial 
# baseline_parameters = ax_client.get_trial_parameters(trial_index=0)
# ax_client.complete_trial(trial_index=0, raw_data=train_evaluate(baseline_parameters))

In [ ]:
# def train_evaluate(parameterization, use_validation_sets = 1):
#     dataloader_batch_size = 50
#     # run_epochs = 2

#     loss_list = []    
#     for validate_i in list(np.random.choice(
#         [i for i in range(len(split_info['validate']))], 
#         use_validation_sets, 
#         replace = False)):
#         print(f"Running with validation set: {validate_i}")

#         val_dict = find_idxs_split_dict(
#             obs_df = obs_df_ref, 
#             split_dict = split_info['validate'][validate_i]
#         )

#         train_idx = val_dict['train_idx']
#         test_idx  = val_dict['test_idx']

#         training_dataloader = DataLoader(
#             ACGTDataset(y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float), 
#                         G = torch.from_numpy(ACGT).to(torch.float), 
#                         idx_original = torch.from_numpy(np.array(train_idx)),
#                         idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
#                         use_gpu_num = 0,
#                         device = 'cuda'
#                        ),
#             batch_size = 50,
#             shuffle = True
#         )

#         testing_dataloader = DataLoader(
#             ACGTDataset(y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float), 
#                         G = torch.from_numpy(ACGT).to(torch.float), 
#                         idx_original = torch.from_numpy(np.array(test_idx)),
#                         idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
#                         use_gpu_num = 0,
#                         device = 'cuda'
#                        ),
#             batch_size = 50,
#             shuffle = True
#         )

#         model = NeuralNetwork(parameterization = parameterization)    
#         model.to(device)    
#         model, loss_df = train_nn(
#             cache_path,
#             training_dataloader,
#             testing_dataloader,
#             model,
#             learning_rate = 1e-3,
#             batch_size = dataloader_batch_size,
#             epochs = parameterization[f"epoch_of_{parameterization['num_layers']}"]
#         )
#         loss_list += [list(loss_df['TestMSE'])[-1]]
    
#     mean_loss = (sum(loss_list)/len(loss_list))
#     print((mean_loss, loss_list))
#     return(mean_loss)

In [ ]:
# train_evaluate(parameterization)

In [ ]:
# for i in range(10):
#     parameters, trial_index = ax_client.get_next_trial()
#     # Local evaluation here can be replaced with deployment to external system.
#     ax_client.complete_trial(trial_index=trial_index, 
#                              raw_data=train_evaluate(parameters))
    
#     # Ax has built in functions to save client state to JSON or to a database. The builtins 
#     # (save_to_json_file, load_from_json_file) fail, seemingly from having a nested search space. 
#     # No debugging information found on Stackoverflow or google so I'm using this work around. 
#     # Warning on running ax_client.complete_trial
#     #     UserWarning:
#     # Cannot flatten observation features ObservationFeatures(parameters={}, trial_index=0) as full parameterization is not recorded in metadata.
    
#     with open(cache_path+'ax_client.pkl', 'wb') as f:
#         pkl.dump(ax_client, f)

In [ ]:
# ax_client.get_trials_data_frame()

In [ ]:
# best_parameters, values = ax_client.get_best_parameters()
# best_parameters

In [ ]:
# mean, covariance = values
# mean

In [ ]:
# render(ax_client.get_contour_plot(param_x="drop_1_of_2", 
#                                   param_y="drop_2_of_2", 
#                                   metric_name="rmse"))

In [ ]:
# ax_client.save_to_json_file(filepath = cache_path+'ax_client.json') 

In [ ]:
# restored_ax_client = (
#     AxClient.load_from_json_file(filepath = cache_path+'ax_client.json') 
# )

## Train Finalized Model

In [ ]:
# dataloader_batch_size = 50
# run_epochs = 2#100

# # don't run if either of these exist because there may be cases where we want the results but not the model
# import re

# if not os.path.exists(cache_path+'/model.pt'): 
#     # Shared setup (train from scratch and load latest)
#     model = NeuralNetwork()

#     # find the biggest model to save
#     saved_models = os.listdir(cache_path)
#     saved_models = [e for e in saved_models if re.match('model*', e)]

#     if saved_models == []:
#         epochs_run = 0
#     else:
#         saved_models = [e for e in saved_models if e != 'model.pt']
#         # if there are saved models reload and resume training
#         saved_models_numbers = [int(e.replace('model_', ''
#                                     ).replace('.pt', ''
#                                     ).split('_')[0]) for e in saved_models]
#         # saved_models
#         epochs_run = max(saved_models_numbers)+1 # add 1 to account for 0 index
#         latest_model = [e for e in saved_models if re.match(
#             '^model_'+str(epochs_run-1)+'_.*\.pt$', e)][0] # subtract 1 to convert back
#         model.load_state_dict(torch.load(cache_path+latest_model))
#         print('Resuming Training: '+str(epochs_run)+'/'+str(run_epochs)+' epochs run.')
    
#     model.to(device)    

#     model, loss_df = train_nn(
#         cache_path,
#         training_dataloader,
#         testing_dataloader,
#         model,
#         learning_rate = 1e-3,
#         batch_size = dataloader_batch_size,
#         epochs = (run_epochs - epochs_run)
#     )
    
#     # experimental outputs:
#     # 1. Model
#     torch.save(model.state_dict(), cache_path+'/model.pt') # convention is to use .pt or .pth

#     # 2. loss_df    
#     # If this is resuming training, load and extend the existing loss dataframe
#     if os.path.exists(cache_path+'/loss_df.csv'):
#         loss_df_on_disk = pd.read_csv(cache_path+'/loss_df.csv')
#         epoch_offset = 1 + loss_df_on_disk['Epoch'].max()
#         loss_df['Epoch'] = loss_df['Epoch'] + epoch_offset
#         loss_df = pd.concat([loss_df_on_disk, loss_df])
#     loss_df.to_csv(cache_path+'/loss_df.csv', index=False)  
    
#     # 3. predictions 
#     yhats = pd.concat([
#         yhat_loop(testing_dataloader, model).assign(Split = 'Test'),
#         yhat_loop(training_dataloader, model).assign(Split = 'Train')], axis = 0)

#     yhats.to_csv(cache_path+'/yhats.csv', index=False)

### Standard Visualizations

In [ ]:
# scale_dict = {'y1':YMat_cs}
# import plotly.graph_objects as go

In [ ]:
# naieve_yhat = training_dataloader.dataset.y.mean()

# naieve_MSE_Train = reverse_cs( 
#     np.array(((naieve_yhat - training_dataloader.dataset.y)**2)).mean(),
#     scale_dict['y1']
# )

# naieve_MSE_Test = reverse_cs( 
#     np.array(((naieve_yhat - testing_dataloader.dataset.y)**2)).mean(),
#     scale_dict['y1']
# )

# naieve_MSE_Train, naieve_MSE_Test



# loss_df = pd.read_csv(cache_path+'/loss_df.csv')

# loss_df.TrainMSE = reverse_cs(loss_df.TrainMSE, scale_dict['y1'])
# loss_df.TestMSE  = reverse_cs(loss_df.TestMSE , scale_dict['y1'])


# fig = go.Figure()
# fig.add_trace(go.Scatter(x=loss_df.Epoch, y=loss_df.TestMSE,
#                     mode='lines', name='Test'))
# fig.add_trace(go.Scatter(x=loss_df.Epoch, y=loss_df.TrainMSE,
#                     mode='lines', name='Train'))

# fig.add_trace(go.Scatter(x=loss_df.Epoch, y=[naieve_MSE_Test  for e in range(len(loss_df.Epoch))], 
#                          mode='lines', name='Naieve Test'))
# fig.add_trace(go.Scatter(x=loss_df.Epoch, y=[naieve_MSE_Train for e in range(len(loss_df.Epoch))], 
#                          mode='lines', name='Naieve Train'))
# fig.show()

In [ ]:
# yhats = pd.read_csv(cache_path+'/yhats.csv')

# # px.scatter(yhats, x = 'y_true', y = 'y_pred', color = 'Split')

In [ ]:
# yhats.y_true = reverse_cs(yhats.y_true, scale_dict['y1'])
# yhats.y_pred = reverse_cs(yhats.y_pred, scale_dict['y1'])

# # px.scatter(yhats, x = 'y_true', y = 'y_pred', color = 'Split', trendline="ols")

In [ ]:
# yhats['Error'] = yhats.y_pred - yhats.y_true

# px.histogram(yhats, x = 'Error', color = 'Split',
#              marginal="box", # can be `rug`, `violin`
#              nbins= 50)

In [ ]:
# os._exit(00)